# Asteroid Hunter - Classify Images

**This code has been created to be run using Gooogle Colab.**
This Code is used to classify images using an already trained model hosted in Google Cloud AutoML Vision (Asteroid Hunter Project).


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/Pablo-GRC/AH-Satellite_Trails/blob/main/Satellite_Detection_v3.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/colab_logo_1x.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/Pablo-GRC/AH-Satellite_Trails/blob/main/Satellite_Detection_v3.ipynb"><img src="https://quantumai.google/site-assets/images/buttons/github_logo_1x.png" />View source on GitHub</a>
  </td>
</table>







## Install required code

We start with a pip install: this package is missing from the default Google Colab library pack.

**Important:** click on "restart runtime" in the cell's output.

In [2]:
!pip install --upgrade google-cloud-automl

     |████████████████████████████████| 389kB 3.9MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 


## Initialize and Authenticate

Now, let's login to Google Cloud using your Gmail account. 

1. Launch the cell below and click on the link. Accept the Google Cloud conditions.
1. Once you have accepted, a login code will be provided. Copy this code and paste it into the case from provided by this case.



In [27]:
from google.cloud import automl
import time
import pandas as pd
import numpy as np
import csv
import json
import os
import shutil
import subprocess
import uuid

from google.colab import auth
from google.colab import files
uniq_id = uuid.uuid4()

auth.authenticate_user()
os.makedirs('/content/images_to_classify/', exist_ok = True)


## Prepare the Service Accout Key 

The AutoML model you will be running is private. The JSON file provided by the research team will allow you to access this model.

This JSON file must be uploaded to the Codelab environment. Run the following code cell and upload the JSON file.

In [2]:
uploaded = files.upload()

Saving hst-asteroid-detection-ae7516235f0b.json to hst-asteroid-detection-ae7516235f0b.json



## Gcloud configuration

Now we configure gcloud to run on the correct Google Cloud Project. That is, the one where the AutoML model is hosted.

In [ ]:
project_id = 'hst-asteroid-detection'
!gcloud config set project {project_id}

## Upload images

Here you can upload the images you want to classify with the AutoML model. Run the following cell and select the files you want to upload from your local computer.

Using the file browser from Colab, verify that the files in the `images_to_classify` are what you chose. 

In [24]:

uploaded = files.upload()
for file in list(uploaded):
  shutil.move(file, "/content/images_to_classify")

Saving idiq10010.jpg to idiq10010.jpg
Saving idjy01030.jpg to idjy01030.jpg


## Prepare the files to be classified

The following cell will prepare the files on a Google Cloud Storage bucket to be processed by the AutoML classifier

In [38]:
#Copy to Google Cloud the files to classify

copy1 = subprocess.Popen('gsutil -m cp -r /content/images_to_classify/* gs://hst-satellites-public/'+uniq_id.hex, shell=True, stdout=subprocess.PIPE)
copy1.wait()

#Create the csv needed by AutoML

images_list = !gsutil ls gs://hst-satellites-public/$uniq_id.hex
the_list = pd.DataFrame(images_list)
the_list.to_csv('images.csv', header=False, index=False)

copy2 = subprocess.Popen('gsutil cp  images.csv gs://hst-satellites-public/'+uniq_id.hex, shell=True, stdout=subprocess.PIPE)
copy2.wait()


0

# Batch Classification

And now we just launch our images classification. 

It will take between 45 minutes (this is the  built-in minimum for AutoML Batch Classification) and several hours depending on the amount of images uploaded.

In [39]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/content/hst-asteroid-detection-ae7516235f0b.json"
os.environ["GOOGLE_CLOUD_PROJECT"]="hst-asteroid-detection"

# Variables
project_id = "129924192384"
model_id= "IOD5322289388342738944"
input_uri = "gs://hst-satellites-public/"+uniq_id.hex+"/images.csv"
output_uri = "gs://hst-satellites-public/output/"+uniq_id.hex

prediction_client = automl.PredictionServiceClient()

starttime = time.time()

    # Get the full path of the model.
model_full_id = f"projects/{project_id}/locations/us-central1/models/{model_id}"

gcs_source = automl.GcsSource(input_uris=[input_uri])

input_config = automl.BatchPredictInputConfig(gcs_source=gcs_source)
gcs_destination = automl.GcsDestination(output_uri_prefix=output_uri)
output_config = automl.BatchPredictOutputConfig(
gcs_destination=gcs_destination
    )

params = {"score_threshold": "0.5"}

response = prediction_client.batch_predict(
    name=model_full_id,
    input_config=input_config,
    output_config=output_config,
    params = params
    )

print("Waiting for operation to complete...")
print(
        f"Batch Prediction results saved to Cloud Storage bucket. {response.result()}"
    )

print('That took {} seconds'.format(time.time() - starttime))


Waiting for operation to complete...
Batch Prediction results saved to Cloud Storage bucket. 
That took 1454.0995559692383 seconds


## Retrieve final data and clean up

Once the classification is finished, we import the results from Google Cloud. After running this next cell, you will find a directory starting with `prediction-HST...` created in your local folder. You can examine the JSON file in this folder to get the classifications for your images.

Inside the JSON file you will find the label, score and the corners of the bounding box for the different objects detected in the uploaded images.



In [40]:
copy1 = subprocess.Popen('gsutil -m cp -r gs://hst-satellites-public/output/'+ uniq_id.hex +'/* . ', shell=True, stdout=subprocess.PIPE)
copy1.wait()

subprocess.Popen('gsutil rm -rf gs://hst-satellites-public/' + uniq_id.hex, shell=True, stdout=subprocess.PIPE)
subprocess.Popen('gsutil rm -rf gs://hst-satellites-public/output/' + uniq_id.hex, shell=True, stdout=subprocess.PIPE)

## Examine the data

It will look something like this:
```
{
   "ID":"gs://hst-satellites-public/12a5a607a7a845a386e50bb85be9d3ca/idjy01030.jpg",
   "annotations":[
      {
         "annotation_spec_id":"4558998520735989760",
         "display_name":"satellite",
         "image_object_detection":{
            "bounding_box":{
               "normalized_vertices":[
                  {
                     "x":0.25213432,
                     "y":0.015150696
                  },
                  {
                     "x":0.66501713,
                     "y":0.9451075799999999
                  }
               ],
               "vertices":[
                  
               ]
            },
            "score":0.9963059399999999
         }
      }
   ]
}
```